In [ ]:
from forex.pre_training_data_prep.config import config
config

## Get previous Pandas dataframe and put it into Spark

In [ ]:
#
# Done
#
#from forex.pre_training_data_prep.tasks.spark.task_convert_pandas_df_to_spark_df import task_convert_pandas_df_to_spark_df
#task_convert_pandas_df_to_spark_df(**config)

In [ ]:
from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_conversion_to_spark'])
sdf_arrays.show(5)
spark.stop()

In [ ]:
#
# Done
#
#from forex.pre_training_data_prep.tasks.spark.array_tasks import task_pivot_and_sort_arrays
#task_pivot_and_sort_arrays(**config)

In [ ]:
#
# Done
#
#from forex.pre_training_data_prep.tasks.spark.array_tasks import task_diff_the_timestamp_arrays
#task_diff_the_timestamp_arrays(**config)

In [ ]:
from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_pivot_and_sort'])
sdf_arrays.show(5)
spark.stop()

In [ ]:
from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_timestamp_diff'])
sdf_arrays.show(5)
spark.stop()

In [ ]:
from pyspark.sql.types import ArrayType, IntegerType, FloatType

In [ ]:
def get_all_timestamps(timestamp_array, seconds_divisor):
    return [int(x) for x in range(min(timestamp_array), max(timestamp_array) + seconds_divisor, seconds_divisor)]

udf_get_all_timestamps = f.udf(get_all_timestamps, ArrayType(IntegerType()))

##https://stackoverflow.com/questions/41190852/most-efficient-way-to-forward-fill-nan-values-in-numpy-array
#def do_nans_exist(values_array):
#    values_array = np.array([np.array(values_array)])
#    mask = np.isnan(values_array)
#    has_nan_0_or_1 = np.max([int(x) for x in mask[0]])
#    return int(has_nan_0_or_1)
#
#udf_do_nans_exist = f.udf(do_nans_exist, IntegerType())

##https://stackoverflow.com/questions/41190852/most-efficient-way-to-forward-fill-nan-values-in-numpy-array
#def do_non_nans_exist(values_array):
#    values_array = np.array([np.array(values_array)])
#    mask = ~np.isnan(values_array)
#    has_non_nan_0_or_1 = np.max([int(x) for x in mask[0]])
#    return int(has_non_nan_0_or_1)
#    
#udf_do_non_nans_exist = f.udf(do_non_nans_exist, IntegerType())


def count_nans_in_array(values_array):
    values_array = np.array([np.array(values_array)])
    mask = np.isnan(values_array)
    nan_count = np.sum([int(x) for x in mask[0]])
    return int(nan_count)
    
udf_count_nans_in_array = f.udf(count_nans_in_array, IntegerType())



##https://stackoverflow.com/questions/41190852/most-efficient-way-to-forward-fill-nan-values-in-numpy-array
#def forward_fill(values_array):
#    arr = np.array([values_array])
#    mask = np.isnan(arr)
#    idx = np.where(~mask, np.arange(mask.shape[1]), 0)
#    np.maximum.accumulate(idx, axis = 1, out = idx)
#    arr[mask] = arr[np.nonzero(mask)[0], idx[mask]]
#    to_return = list([float(x) for x in arr[0]])
#    return to_return
#
#udf_forward_fill = f.udf(forward_fill, ArrayType(FloatType()))


def locate_nans(timestamp_array, timestamp_all_array, values_array):

    # make sure we get an argsort in here later to ensure order of values is correct

    ts = np.array(timestamp_array, dtype = np.uint64) # ??
    ts_all = np.array(timestamp_all_array, dtype = np.uint64)  # we can probably make this smaller
    v = np.array(values_array, dtype = np.float64)  # we can probably make this smaller
    
    pdf = pd.DataFrame({'timestamp' : ts, 'values' : v})
    pdf_all = pd.DataFrame({'timestamp' : ts_all})

    pdf_joined = (
        pd.merge(
            pdf_all,
            pdf,
            on = 'timestamp',
            how = 'left',
        )
    )

    to_return = pdf_joined['values'].to_list()
    
    return to_return

udf_locate_nans = f.udf(locate_nans, ArrayType(FloatType()))

In [ ]:
import pyspark.sql.functions as f

from utilities.spark_session import get_spark_session

from forex.pre_training_data_prep.tasks.spark.nan_related_tasks import udf_get_all_timestamps
from forex.pre_training_data_prep.tasks.spark.nan_related_tasks import udf_locate_nans
from forex.pre_training_data_prep.tasks.spark.nan_related_tasks import udf_count_nans_in_array

def task_find_full_day_nans(**config):

    spark = get_spark_session(config['spark_config'])
    sdf_arrays = (
        spark.read.parquet(config['directory_output'] + '/' + config['filename_timestamp_diff'])
        .coalesce(config['n_processors_to_coalesce'])
        .orderBy('date_post_shift')
        .withColumn(
            'timestamps_all',
            udf_get_all_timestamps(f.col('sorted_timestamp_array'), f.lit(config['seconds_divisor']))
        )
    )
    
    for item in config['list_data_columns']:
        sdf_arrays = (
            sdf_arrays
            .withColumn(
                item + '_and_nans',
                udf_locate_nans(f.col('sorted_timestamp_array'), f.col('timestamps_all'), f.col('sorted_' + item + '_array'))
            )
        )

    for item in config['list_data_columns']:
        sdf_arrays = (
            sdf_arrays
            .withColumn(
                item + '_nan_count',
                udf_count_nans_in_array(f.col(item + '_and_nans'))
            )
        )

    sdf_arrays = sdf_arrays.withColumn('nan_count_full_day', f.col('return_nan_count'))
    for item in config['list_data_columns']:
        sdf_arrays = sdf_arrays.drop(item + '_nan_count')

    sdf_arrays.write.mode('overwrite').parquet(config['directory_output'] + '/' + config['filename_full_day_nans'])
    spark.stop()

In [ ]:
from forex.pre_training_data_prep.tasks.spark.nan_related_tasks import task_find_full_day_nans
task_find_full_day_nans(**config)

In [ ]:
from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_full_day_nans'])
sdf_arrays.show(5)
spark.stop()

In [ ]:
# stopped here for now

In [ ]:
## QA #1

In [ ]:
config['directory_output'] = config['directory_output'] + '/manual__2025-03-19T08:37:57.595997+00:00'

In [ ]:
import pyspark.sql.functions as f
import numpy as np
from utilities.spark_session import get_spark_session

In [ ]:
spark = get_spark_session(config['spark_config'])
sdf_qa = (
    spark.read.parquet(config['directory_output'] + '/' + config['filename_full_day_nans'])
    .select('nan_count_full_day', 'date_post_shift')
    .orderBy(f.col('nan_count_full_day').desc())
    .withColumn('day_has_nans', f.col('nan_count_full_day') > 0)
    .groupBy('day_has_nans').agg(f.count('date_post_shift').alias('number_of_days'))
)
sdf_qa.show()

In [ ]:
spark = get_spark_session(config['spark_config'])
sdf_qa_counts = (
    spark.read.parquet(config['directory_output'] + '/' + config['filename_full_day_nans'])
    .select('nan_count_full_day', 'date_post_shift')
    .orderBy(f.col('nan_count_full_day').desc())
)
sdf_qa_counts.show()

In [ ]:
import matplotlib.pyplot as plt
nan_counts_per_day = [x['nan_count_full_day'] for x in sdf_qa_counts.select('nan_count_full_day').collect()]

#plt.figure()
#plt.hist(nan_counts_per_day)
#plt.show()
#plt.close()

plt.figure()
plt.boxplot([nan_counts_per_day], widths=0.95)
plt.show()
plt.close()

In [ ]:
print(np.mean(nan_counts_per_day), np.median(nan_counts_per_day))

In [ ]:
pdf_qa_dates = sdf_qa_counts.toPandas()

In [ ]:
pdf_qa_dates['weekday'] = [x.weekday() for x in pdf_qa_dates['date_post_shift']]

pdf_qa_dates.boxplot(by = 'weekday')
plt.show()
plt.close()

In [ ]:
## end QA #1

## Investigate consecutive NaNs for each day

By identifying the maximum number of NaNs in a row for a given day:

In [ ]:
from forex.pre_training_data_prep.config import config
from utilities.spark_session import get_spark_session

run_id = 'manual__2025-03-19T08:37:57.595997+00:00'  # TEMP

spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + run_id + '/' + config['filename_full_day_nans'])

stuff_1 = sdf_arrays.columns

sdf_arrays.show(3)

In [ ]:
#import matplotlib.pyplot as plt

#from itertools import groupby

import pyspark.sql.functions as f
#from pyspark.sql.types import BooleanType


from utilities.spark_session import get_spark_session

def booger(**config):

    from forex.pre_training_data_prep.tasks.spark.nan_related_tasks import udf_get_max_consecutive_NaNs
    from utilities.test_all_equality import udf_test_all_equality
    
    spark = get_spark_session(config['spark_config'])
    run_id = config['dag_run'].run_id
    sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + run_id + '/' + config['filename_full_day_nans'])

    for item in config['list_data_columns']:
        sdf_arrays = sdf_arrays.withColumn('max_consec_nan_' + item, udf_get_max_consecutive_NaNs(f.col(item + '_and_nans')))

    columns_to_test = [f.col('max_consec_nan_' + x) for x in config['list_data_columns']]
    sdf_arrays = sdf_arrays.withColumn('is_consec_equal', udf_test_all_equality(*columns_to_test))

    sdf_arrays = (
        sdf_arrays
        .withColumn('max_daily_consec_nans', f.col('max_consec_nan_' + config['list_data_columns'][0]))
    )

    # clean up
    sdf_arrays = sdf_arrays.drop('timestamps_all')
    for item in config['list_data_columns']:
        sdf_arrays = (
            sdf_arrays
            .drop(
                #'max_consec_nan_' + item,
                item + '_and_nans',
            )
        )
    spark.stop()                                       


booger(**config)




#columns_to_select = ['date_post_shift', 'nan_count_full_day']
#columns_to_select.extend(['max_consec_nan_' + x for x in config['list_data_columns']])
#columns_to_select.append('is_consec_equal')
#
#sdf_temp = sdf_arrays.select(*columns_to_select)
#
##sdf_temp.show(5)
#
## should all be true
#sdf_agg = (
#    sdf_temp.select('date_post_shift', 'is_consec_equal')
#    .groupBy('is_consec_equal')
#    .agg(f.count('date_post_shift').alias('count_column'))
#)
##sdf_agg.show()
#
#
##sdf_arrays = (
##    sdf_arrays
##    .withColumn('max_daily_consec_nans', f.col('max_consec_nan_' + config['list_data_columns'][0]))
##    .drop('is_consec_equal')
##    .drop('sorted_timestamp_array', 'diff_sorted_timestamp_array')
##)
#
##for item in config['list_data_columns']:
##    sdf_arrays = (
##        sdf_arrays
##        .drop(
##            'max_consec_nan_' + item,
##            'sorted_' + item + '_array',
##        )
##    )
#
#
##
## length tests
##
#sdf_arrays = sdf_arrays.withColumn('len_timestamps_all', f.array_size(f.col('timestamps_all')))
#for item in config['list_data_columns']:
#    sdf_arrays = sdf_arrays.withColumn('len_' + item, f.array_size(f.col(item + '_and_nans')))
#
#columns_to_test = [f.col('len_' + x) for x in config['list_data_columns']]
#columns_to_test.extend(['len_timestamps_all'])
#
#sdf_arrays = sdf_arrays.withColumn('is_length_equal', udf_test_all_equality(*columns_to_test))
#
## True
#sdf_agg = (
#    sdf_arrays.select('date_post_shift', 'is_length_equal')
#    .groupBy('is_length_equal')
#    .agg(f.count('date_post_shift').alias('count_column'))
#)
##sdf_agg.show()
#
#sdf_arrays = sdf_arrays.drop('is_length_equal', 'len_timestamps_all')
#for item in config['list_data_columns']:
#    sdf_arrays = sdf_arrays.drop('len_' + item)
#
#sdf_arrays.show(3)
#
#sdf_arrays.drop('nan_count_full_day', 'max_daily_consec_nans').write.mode('overwrite').parquet(config['directory_output'] + '/' + 'PLACEHOLDER.parquet')
#

#sdf_arrays = sdf_arrays.drop('nan_count_full_day', 'max_daily_consec_nans')




In [ ]:
from forex.pre_training_data_prep.config import config

from utilities.spark_session import get_spark_session
run_id = 'manual__2025-03-21T03:30:18.253565+00:00'  # temp
spark = get_spark_session(config['spark_config'])

sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + run_id + '/' + config['filename_full_day_nans'])
print(sdf_arrays.columns)
print()
sdf_arrays.show(3)
print()

sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + run_id + '/QA/' + config['filename_qa_day_nan_counts'])
print(sdf_arrays.columns)
print()
sdf_arrays.show(3)
print()

#print()
#print(config['filename_qa_full_day_consecutive_nans'])
#print()

sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + run_id + '/QA/' + config['filename_qa_full_day_consecutive_nans'])
print(sdf_arrays.columns)
print()
sdf_arrays.show(3)
print()

spark.stop()




In [ ]:
stuff_2 = sdf_arrays.columns
print(stuff_1)
print()
print(stuff_2)

## Trig functions

In [ ]:
from forex.pre_training_data_prep.tasks.spark.trig import add_trig
add_trig(**config)

## Window

#### Ensure there is room

In [ ]:
from forex.pre_training_data_prep.tasks.spark.spark_sliding_window import test_window_space
test_window_space(**config)

#### Compute the sliding windows

In [ ]:
from forex.pre_training_data_prep.tasks.spark.spark_sliding_window import do_sliding_window
do_sliding_window(**config)

In [19]:
from forex.pre_training_data_prep.config import config
run_id = 'manual__2025-03-21T04:16:18.017637+00:00'

from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])

sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + run_id + '/spark_sliding_window_space_check.parquet')
sdf_arrays.show(3)
print()

sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + run_id + '/QA/spark_sliding_window_QA.parquet')
sdf_arrays.show(3)
print()

sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + run_id + '/' + config['filename_sliding_window'])
sdf_arrays.show(3)

spark.stop()

+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|date_post_shift|      timestamps_all|     return_and_nans| volatility_and_nans|     volume_and_nans|   lhc_mean_and_nans|              sin_24|              cos_24|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     2021-05-14|[1620939660, 1620...|[0.0, -3.9E-4, 1....|[0.0, 4.6E-4, 1.1...|[1.0, 6.0, 2.0, N...|[1.20806, 1.20789...|[-0.7040147, -0.7...|[0.71018535, 0.71...|
|     2021-05-17|[1621198800, 1621...|[-2.4E-4, -5.9E-4...|[2.4E-4, 5.9E-4, ...|[4.0, 4.0, 4.0, N...|[1.21458, 1.21419...|[-0.70710677, -0....|[0.70710677, 0.71...|
|     2021-05-18|[1621285260, 1621...|[0.0, -6.0E-5, Na...|[0.0, 6.0E-5, NaN...|[1.0, 3.0, NaN, N...|[1.21528, 1.21524...|[-0.7040147, -0.7...|[0.71018535, 0.71...|
+---------

In [ ]:
#from forex.pre_training_data_prep.config import config
#from utilities.spark_session import get_spark_session
#spark = get_spark_session(config['spark_config'])
#sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_sliding_window_space_check'])
#sdf_arrays.show(3)

In [ ]:
# test sliding window

In [25]:
from forex.pre_training_data_prep.config import config
run_id = 'manual__2025-03-21T04:16:18.017637+00:00'

from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + run_id + '/' + config['filename_sliding_window'])
sdf_arrays.show(3)

+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|date_post_shift|        sw_timestamp|           sw_return|       sw_volatility|           sw_volume|         sw_lhc_mean|           sw_sin_24|           sw_cos_24|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     2016-04-13|[[1460494860, 146...|[[4.0E-5, -1.2E-4...|[[4.0E-5, 2.0E-4,...|[[2.0, 13.0, 3.0,...|[[1.1384866, 1.13...|[[-0.7040147, -0....|[[0.71018535, 0.7...|
|     2016-04-14|[[1460581260, 146...|[[3.0E-5, NaN, Na...|[[3.0E-5, NaN, Na...|[[2.0, NaN, NaN, ...|[[1.12739, NaN, N...|[[-0.7040147, -0....|[[0.71018535, 0.7...|
|     2016-04-15|[[1460667660, 146...|[[0.0, 2.0E-5, -5...|[[2.0E-5, 6.0E-5,...|[[4.0, 4.0, 8.0, ...|[[1.12635, 1.1263...|[[-0.7040147, -0....|[[0.71018535, 0.7...|
+---------

In [26]:
import pyspark.sql.functions as f
from pyspark.sql.types import IntegerType, FloatType

def testI(a_list_of_lists):
    i = a_list_of_lists[0][0]
    return i

def testJ(a_list_of_lists):
    j = a_list_of_lists[20][0]
    return j

udf_test_int_i = f.udf(testI, IntegerType())
udf_test_int_j = f.udf(testJ, IntegerType())

udf_test_float_i = f.udf(testI, FloatType())
udf_test_float_j = f.udf(testJ, FloatType())

In [22]:
(
    sdf_arrays
    .withColumn('I', udf_test_int_i(f.col('sw_timestamp')))
    .withColumn('J', udf_test_int_j(f.col('sw_timestamp')))
    .select('date_post_shift', 'I', 'J')
    .show(3)
)

+---------------+----------+----------+
|date_post_shift|         I|         J|
+---------------+----------+----------+
|     2016-04-13|1460494860|1460506860|
|     2016-04-14|1460581260|1460593260|
|     2016-04-15|1460667660|1460679660|
+---------------+----------+----------+
only showing top 3 rows



In [23]:
(
    sdf_arrays
    .withColumn('I', udf_test_float_i(f.col('sw_return')))
    .withColumn('J', udf_test_float_j(f.col('sw_return')))
    .select('date_post_shift', 'I', 'J')
    .show(3)
)

+---------------+------+-------+
|date_post_shift|     I|      J|
+---------------+------+-------+
|     2016-04-13|4.0E-5| 6.0E-5|
|     2016-04-14|3.0E-5|-6.0E-5|
|     2016-04-15|   0.0|-6.0E-5|
+---------------+------+-------+
only showing top 3 rows



In [24]:
(
    sdf_arrays
    .withColumn('I', udf_test_float_i(f.col('sw_sin_24')))
    .withColumn('J', udf_test_float_j(f.col('sw_sin_24')))
    .select('date_post_shift', 'I', 'J')
    .show(3)
)

+---------------+----------+-----------+
|date_post_shift|         I|          J|
+---------------+----------+-----------+
|     2016-04-13|-0.7040147|0.091501616|
|     2016-04-14|-0.7040147|0.091501616|
|     2016-04-15|-0.7040147|0.091501616|
+---------------+----------+-----------+
only showing top 3 rows



In [27]:
(
    sdf_arrays
    .withColumn('I_sw_timestamp', udf_test_int_i(f.col('sw_timestamp')))
    .withColumn('J_sw_timestamp', udf_test_int_j(f.col('sw_timestamp')))
    .withColumn('I_sw_return', udf_test_float_i(f.col('sw_return')))
    .withColumn('J_sw_return', udf_test_float_j(f.col('sw_return')))
    .withColumn('I_sw_sin_24', udf_test_float_i(f.col('sw_sin_24')))
    .withColumn('J_sw_sin_24', udf_test_float_j(f.col('sw_sin_24')))
    .select(
        'date_post_shift',
        'I_sw_timestamp', 'J_sw_timestamp',
        'I_sw_return', 'J_sw_return',
        'I_sw_sin_24', 'J_sw_sin_24',
    )
    .show(3)
)

+---------------+--------------+--------------+-----------+-----------+-----------+-----------+
|date_post_shift|I_sw_timestamp|J_sw_timestamp|I_sw_return|J_sw_return|I_sw_sin_24|J_sw_sin_24|
+---------------+--------------+--------------+-----------+-----------+-----------+-----------+
|     2016-04-13|    1460494860|    1460506860|     4.0E-5|     6.0E-5| -0.7040147|0.091501616|
|     2016-04-14|    1460581260|    1460593260|     3.0E-5|    -6.0E-5| -0.7040147|0.091501616|
|     2016-04-15|    1460667660|    1460679660|        0.0|    -6.0E-5| -0.7040147|0.091501616|
+---------------+--------------+--------------+-----------+-----------+-----------+-----------+
only showing top 3 rows



## Expand arrays

In [ ]:
#from forex.pre_training_data_prep.config import config
#config

In [ ]:
#from forex.pre_training_data_prep.tasks.spark.array_expand import task_expand_arrays
#task_expand_arrays(**config)

In [31]:
from utilities.spark_session import get_spark_session
run_id = 'manual__2025-03-22T03:30:14.026683+00:00'
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + run_id + '/' + config['filename_explode_array'])
sdf_arrays.show(3)

+---------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|date_post_shift| timestamp|              return|          volatility|              volume|            lhc_mean|              sin_24|              cos_24|
+---------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     2014-12-15|1418604600|[6.0E-5, -2.0E-4,...|[1.8E-4, 2.2E-4, ...|[21.0, 20.0, 34.0...|[1.2457334, 1.245...|[0.21643962, 0.22...|[0.976296, 0.9753...|
|     2014-12-15|1418605200|[5.0E-5, -2.0E-5,...|[1.3E-4, 1.3E-4, ...|[11.0, 22.0, 18.0...|[1.24569, 1.24575...|[0.25881904, 0.26...|[0.9659258, 0.964...|
|     2014-12-15|1418605800|[-1.2E-4, -4.0E-5...|[1.7E-4, 4.0E-5, ...|[14.0, 8.0, 12.0,...|[1.2454033, 1.245...|[0.3007058, 0.304...|[0.95371693, 0.95...|
+---------------+----------+--------------------+--------------------+

In [32]:
sdf_arrays.count()        

523511

## Find NaNs and decide what to do with them

In [ ]:
from forex.pre_training_data_prep.config import config
config

In [ ]:
from forex.pre_training_data_prep.tasks.spark.nan_related_tasks import udf_count_nans_in_array
#from forex.pre_training_data_prep.tasks.spark.nan_related_tasks import udf_count_nans_in_array
# need something about first list item

In [ ]:
import pyspark.sql.functions as f

from forex.pre_training_data_prep.tasks.spark.nan_related_tasks import udf_get_max_consecutive_NaNs

from utilities.spark_session import get_spark_session

def deal_with_post_sw_nans(**config):
    spark = get_spark_session(config['spark_config'])
    sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_explode_array'])

    items_to_check_list = config['list_data_columns'].copy()
    items_to_check_list.extend(config['list_data_columns_no_scale'])
    
    for item in items_to_check_list:
        sdf_arrays = sdf_arrays.withColumn('max_consec_nan_' + item, udf_get_max_consecutive_NaNs(f.col(item)))

    # We are skipping the QA equality test(s) here. Add it later.

    # is first item NaN?
    # drop if yes
    sdf_arrays = (
        sdf_arrays
        .withColumn('is_first_item_a_nan', f.isnan(f.col('return').getItem(0)))
        .where(~f.col('is_first_item_a_nan'))
        .drop('is_first_item_a_nan')
    )

    # max consecutive NaN count
    # decide on a threshold value after plotting the distribution
    sdf_arrays = (
        sdf_arrays
        .withColumn('max_consec_nans', f.col('max_consec_nan_return'))
        .drop(*['max_consec_nan_' + x for x in items_to_check_list])
    )

    # Again, we are skipping the QA equality test(s) here. Add it later.
    for item in items_to_check_list:
        sdf_arrays = (
            sdf_arrays
            .withColumn(
                item + '_total_nan_count',
                udf_count_nans_in_array(f.col(item))
            )
        )
    sdf_arrays = (
        sdf_arrays
        .withColumn('total_nan_count', f.col(config['list_data_columns'][0] + '_total_nan_count'))
        .drop(*[x + '_total_nan_count' for x in items_to_check_list])
    )

    sdf_arrays.write.mode('overwrite').parquet(config['directory_output'] + '/' + config['filename_post_sw_nans'])    
    spark.stop()

deal_with_post_sw_nans(**config)

In [ ]:
deal_with_post_sw_nans(**config)

## Filter by NaN counts

In [ ]:
from forex.pre_training_data_prep.config import config
config

In [ ]:
from forex.pre_training_data_prep.tasks.spark.nan_related_tasks import plot_post_sw_nans
plot_post_sw_nans(**config)

In [ ]:
from forex.pre_training_data_prep.tasks.spark.nan_related_tasks import filter_by_nan_counts
filter_by_nan_counts(**config)

## Compute mean and standard deviation

(Before forward-fill; after implementing the forward-fill operation we'll perform the actual scaling procedure).

In [ ]:
from forex.pre_training_data_prep.config import config
config

In [ ]:
from forex.pre_training_data_prep.tasks.spark.scale_it_all import compute_scaling_statistics_for_later_use
compute_scaling_statistics_for_later_use(**config)

In [ ]:
from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_scaling_stats'])
sdf_arrays.show(3)
spark.stop()

## Forward fill

In [ ]:
from forex.pre_training_data_prep.config import config
config

In [ ]:
from forex.pre_training_data_prep.tasks.spark.nan_related_tasks import forward_fill_it_all
forward_fill_it_all(**config)

In [ ]:
from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_forward_filled'])
sdf_arrays.show(3)
spark.stop()

## Properly scaled

In [ ]:
from forex.pre_training_data_prep.config import config

In [ ]:
from forex.pre_training_data_prep.tasks.spark.scale_it_all import properly_scale_it_all
properly_scale_it_all(**config)

In [ ]:
from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_scaled'])
sdf_arrays.show(3)
spark.stop()

In [ ]:
from forex.pre_training_data_prep.config import config

In [ ]:
import pandas as pd
import numpy as np
import pickle

from utilities.spark_session import get_spark_session

def stuff(**config):
    np.random.seed(config['shuffle_random_seed'])
    
    dict_results = {}
    
    pdf = pd.read_parquet(config['directory_output'] + '/' + config['filename_scaled'])
    pdf.columns = [x.replace('_scaled', '').replace('_ff', '') for x in pdf.columns]
    
    item_list = []
    item_list.extend(config['list_data_columns'])
    item_list.extend(config['list_data_columns_no_scale'])

    dict_results['features'] = {
        'features_list' : item_list,
        'features_index_lookup' : {
            'name_to_index' : {},
            'index_to_name' : {},
        },
    }
    for i, item in enumerate(item_list):
        dict_results['features']['features_index_lookup']['index_to_name'][i] = item
        dict_results['features']['features_index_lookup']['name_to_index'][item] = i

    # ensure we have the columns completely named how we want them
    new_column_names = ['date_post_shift', 'timestamp']
    new_column_names.extend(item_list)
    new_column_names.extend(['mean_' + item for item in item_list if item in config['list_data_columns']])
    new_column_names.extend(['std_' + item for item in item_list if item in config['list_data_columns']])

    pdf = pdf[new_column_names]
    pdf.sort_values(by = ['timestamp'], inplace = True)  # I probably did this earlier but don't want to fuss with the matter right now

    dict_results['list_of_timestamps'] = [int(x) for x in pdf['timestamp'].values]
    dict_results['list_of_shifted_dates'] = [str(x) for x in pdf['date_post_shift'].values]

    n_back = config['n_back']
    n_forward = config['n_forward']

    M_array = pdf[item_list].to_numpy()
    n_rows, n_features = M_array.shape
    
    # I'm sure there is a more elegant way to do this--without an explicitly programmed loop, that is
    #
    M_Xy = np.empty([n_rows, n_back + n_forward, n_features])
    for row in range(0, n_rows):
        for feature_track in range(0, n_features):
            M_Xy[row, :, feature_track] = M_array[row, feature_track]

    # HARD CODED:  Should be optional
    if True:
        n_step_to_use_here = 5  # should be in config
        M_Xy = M_Xy[::n_step_to_use_here, :, :]   # see if we use n_step any where else, as its use here might not be correct
        
    # this assumes we want chronological order
    indices_train = np.uint64(np.arange(0, np.uint64(np.floor(np.round(M_Xy.shape[0] * config['train_val_test_split'][0])))))
    indices_val = np.uint64(np.arange(indices_train[-1] + 1, np.uint64(np.floor(np.round(indices_train[-1] + M_Xy.shape[0] * config['train_val_test_split'][1])))))
    indices_test = np.uint64(np.arange(indices_val[-1] + 1, np.uint64(np.floor(np.round(indices_val[-1] + M_Xy.shape[0] * config['train_val_test_split'][2])))))

    if config['shuffle_X_train_and_val']:
        np.random.shuffle(indices_train)
        np.random.shuffle(indices_val)
        np.random.shuffle(indices_test)

    M_Xy_train = M_Xy[indices_train, :, :]
    M_Xy_val = M_Xy[indices_val, :, :]
    M_Xy_test = M_Xy[indices_test, :, :]

    #
    # the following could have been condensed into one for loop, but I decided for ease of debugging to code this operation very specifically
    #
    dict_results['matrices'] = {'train' : {}, 'val' : {}, 'test' : {}}
    dict_results['matrices']['train'] = {
        'X' : M_Xy_train[:, 0:n_back, :],
        'y' : M_Xy_train[:, n_back:(n_back + n_forward), :],
    }
    dict_results['matrices']['val'] = {
        'X' : M_Xy_val[:, 0:n_back, :],
        'y' : M_Xy_val[:, n_back:(n_back + n_forward), :],
    }
    dict_results['matrices']['test'] = {
        'X' : M_Xy_test[:, 0:n_back, :],
        'y' : M_Xy_test[:, n_back:(n_back + n_forward), :],
    }
    
    with open(config['directory_output'] + '/' + config['filename_numpy_final_dict'], 'wb') as fff:
        pickle.dump(dict_results, fff)
    
    # save the reorganized pandas dataframe
    pdf.to_parquet(config['directory_output'] + '/' + config['filename_final_pandas_df'])


stuff(**config)

In [ ]:
#with open(config['directory_output'] + '/' + config['filename_numpy_final_dict'], 'rb') as fff:
#    dict_results = pickle.load(fff)

#print(dict_results[

In [ ]:
# blah----9101c2e0-43cb-4a70-9aff-8680f73edf62  -- first run with this code, all of y predicted
# blah----de18c0e6-8907-4c4b-ac9a-1b1c96e4b477  -- second run with this code, mean of y predicted